In [1]:
import numpy as np
import pandas as pd

In [24]:
titanic_df = pd.read_csv('./data/titanic/train.csv')

In [25]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### PassengerId : 탑승자 일련번호
### survived : 생존 여부 (0: 사망, 1: 생존)
### pClass: 선실등급 (1: 일등석, 2: 이등석, 3: 삼등석)
### SibSp: 같이 탑승한 형제자매 또는 배우자 인원수
### Parch: 같이 탑승한 부모님 또는 어린이 인원수
### Ticket: 티켓번호
### Fare: 요금
### Cabin: 선실번호
### embarken: 중간 정착 항구 C = Cherbourg, Q = Queenstown, S = Southampton

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [27]:
# 생존 여부를 예측하는 것이므로 y로 따로 빼줌
y_titanic = titanic_df.Survived
x_titanic = titanic_df.drop('Survived', axis=1)

In [28]:
# 각 특성 확인 (결측치 및 타입)
x_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [29]:
# 카테고리컬 피처를 숫자로 변환해줌
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x_titanic['Sex'] = le.fit_transform(x_titanic.Sex)

In [30]:
# 필요없는 특성 삭제
x_titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [33]:
# Age 결측치 => Age의 평균으로 처리
x_titanic.Age.fillna(x_titanic.Age.mean(), inplace=True)

In [35]:
# Embarked 결측치 =>  N으로 처리
x_titanic.Embarked.fillna('N', inplace=True)

In [36]:
x_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int32  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    object 
dtypes: float64(2), int32(1), int64(3), object(1)
memory usage: 45.4+ KB


In [38]:
# Embarked 특성 One-hot-Encoding
x_titanic = pd.get_dummies(x_titanic)

In [39]:
x_titanic.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_N,Embarked_Q,Embarked_S
0,3,1,22.0,1,0,7.2500,0,0,0,1
1,1,0,38.0,1,0,71.2833,1,0,0,0
2,3,0,26.0,0,0,7.9250,0,0,0,1
3,1,0,35.0,1,0,53.1000,0,0,0,1
4,3,1,35.0,0,0,8.0500,0,0,0,1


In [41]:
# GridSearchCV : 하이퍼파리미터 튜닝과 CV (교차검증) 까지 해줌
from sklearn.model_selection import GridSearchCV

dt = DecisionTreeClassifier()

params = {
    'max_depth' : [2, 3, 4, 5],
    'min_samples_split' : [3, 4, 5],
    'min_samples_leaf': [2, 3]
}

grid = GridSearchCV(dt, param_grid=params, cv=5, refit=True)

In [46]:
# 데이터 스플릿 (기본 3:1 비율로 나눔)
train_x, test_x, train_y, test_y = train_test_split(x_titanic, y_titanic)

In [49]:
# 학습
grid.fit(train_x, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [2, 3, 4, 5], 'min_samples_leaf': [2, 3]

In [50]:
# 점수
grid.score(test_x, test_y)

0.8340807174887892